# Calculate magnetic parameters

grogupy is specifically written in a modular way, to fully take advantage of python classes. In this simple example we show how to use grogupy to simulate the magnetic interaction between two atoms from a density functional theory (DFT) calculation and create an output file for [magnopy](https://docs.magnopy.org/en/latest/user-guide/start/about.html). In this example we will use a non-collinear Siesta calculation of the Fe3GeTe2 system with spin-orbit coupling.

First import grogupy, which will import the most important classes, functions and variables in its namespace.

In [1]:
import grogupy

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/_tqdm.py:32: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Then we start to create the basic classes based on the desired parameters. First create the ``Kspace`` class which contains the parameters fot the Brillouin zone integration. Because in this case we only take a single layer of Fe3GeTe2 and there is a large vacuum in the perpendicular direction it is sufficient to only 
integrate in the plane of the material. Furthermore for a fast initial calculation a 10x10 grid of k-points should be enough.

In [2]:
Fe3GeTe2_kspace = grogupy.Kspace(kset=[10, 10, 1])
Fe3GeTe2_kspace

<grogupy.Kspace kset=[10 10  1], NK=100>

Then we would like to set the parameters for the complex integral for the Green's 
function calculation. This can be done through the ``Contour`` class. Again, for an 
initial calculation 300 sample points should be enough. The energy minimum should be 
set below the smallest eigenvalue from the Siesta Hamiltonian.


In [3]:
Fe3GeTe2_contour = grogupy.Contour(emin=-15, eset=300, esetp=10000)
Fe3GeTe2_contour

<grogupy.Contour emin=-20, emax=0, eset=300, esetp=10000>

Then we can create the ``Hamiltonian`` class, which extracts and stores the Hamiltonian 
and geometrical data from the Siesta files using the ``sisl`` library. Furthermore we must 
provide the exchange field orientation in the DFT calculation, which is usually the 
perpendicular direction from the 2D material.


In [4]:
Fe3GeTe2_hamiltonian = grogupy.Hamiltonian(
    infile="./../../../benchmarks/Fe3GeTe2/Fe3GeTe2.fdf",
    scf_xcf_orientation=[0, 0, 1],
)
Fe3GeTe2_hamiltonian

Setting up Hamiltonian:   0%|          | 0/243 [00:00<?, ?it/s]

<grogupy.Hamiltonian scf_xcf_orientation=[0 0 1], orientation=[0 0 1], NO=168>

After these initial steps we can create the ``Builder`` class and initialize with the 
above information. We have to provide the directions of the rotated exchange field and the 
two perpendicular directions are calculated automatically.


In [5]:
orientations = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

Fe3GeTe2 = grogupy.Builder(ref_xcf_orientations=orientations)
Fe3GeTe2.add_kspace(Fe3GeTe2_kspace)
Fe3GeTe2.add_contour(Fe3GeTe2_contour)
Fe3GeTe2.add_hamiltonian(Fe3GeTe2_hamiltonian)
Fe3GeTe2

<grogupy.Builder npairs=0, numk=100, kset=[10 10  1], eset=300>

Then we have to define the magnetic entities, which can be a list of orbitals from the 
Siesta Hamiltonian. We use ``sisl`` functions to extract this information based on a more 
human like definition. For example take two shells from two Fe atoms.


In [6]:
magnetic_entities = [dict(atom=3, l=2), dict(atom=4, l=2)]

With this information we can create two ``MagneticEntity`` instances.


In [7]:
Fe3GeTe2.add_magnetic_entities(magnetic_entities)

Add magnetic entities:   0%|          | 0/2 [00:00<?, ?it/s]

This would be enough to calculate the anisotropy on both sites, but it is not 
enough to build a spin model, for this we need the exchange tensor between all the pairs. 
Generally this can be done in a very similar way. First a list of dictionaries must be 
created, where each dictionary contains ``ai``, ``aj`` and ``Ruc``, where the first two is the 
index from the magnetic_entities and the third is the supercell shift of the second 
magnetic entity.


In [8]:
pairs = [
    dict(ai=0, aj=1, Ruc=[0, 0, 0]),
    dict(ai=0, aj=1, Ruc=[1, 0, 0]),
]
Fe3GeTe2.add_pairs(pairs)
Fe3GeTe2

Add pairs:   0%|          | 0/2 [00:00<?, ?it/s]

<grogupy.Builder npairs=2, numk=100, kset=[10 10  1], eset=300>

Now every information is contained in the Fe3GeTe2 instance to run the simulation. 
There are multiple parameters that can be changed to tune the runtime and the precision 
of the simulation, but for now let us use the default parameters.

In [9]:
Fe3GeTe2.solve()

Extracting exchange field:   0%|          | 0/165 [00:00<?, ?it/s]

Rotating Exchange field:   0%|          | 0/3 [00:00<?, ?it/s]

Rotation 1:   0%|          | 0/100 [00:00<?, ?it/s]

Extracting exchange field:   0%|          | 0/165 [00:00<?, ?it/s]

Setup perturbations for rotated hamiltonian:   0%|          | 0/2 [00:00<?, ?it/s]

...

Setup perturbations for rotated hamiltonian:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting exchange field:   0%|          | 0/165 [00:00<?, ?it/s]

Setup perturbations for rotated hamiltonian:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting exchange field:   0%|          | 0/165 [00:00<?, ?it/s]

Setup perturbations for rotated hamiltonian:   0%|          | 0/2 [00:00<?, ?it/s]

Now the instance parameters are filled with the data from the simulation, so we can 
output the information. We can print the instance ``Fe3GeTe2`` to get the information of 
the run and the ``to_magnopy()`` method returns the results in ``magnopy``'s input format.


In [10]:
print(Fe3GeTe2)

grogupy version: 0.1.0
Input file: ./../../../benchmarks/Fe3GeTe2/Fe3GeTe2.fdf
Spin mode: SPIN-ORBIT
Number of orbitals: 168
SLURM job ID: Could not be determined.
Architecture: CPU
Number of threads in the parallel cluster: 1
Parallelization is over: Nothing
Solver used for Greens function calculation: Parallel
Maximum number of Greens function samples per batch: 300
Spin model: generalised-grogu
Cell [Ang]:
3.791001511088653242e+00 0.000000000000000000e+00 0.000000000000000000e+00
-1.895500755544326621e+00 3.283103614407953064e+00 0.000000000000000000e+00
0.000000000000000000e+00 0.000000000000000000e+00 2.057000819825037041e+01
DFT axis: [0 0 1]
Quantization axis and perpendicular rotation directions:
[1 0 0] --> [[ 0.          0.         -1.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.70710678 -0.70710678]]
[0 1 0] --> [[ 1.          0.          0.        ]
 [ 0.          0.         -1.        ]
 [ 0.70710678  0.         -0.70710678]]
[0 0 1] --> [[1.         0

In [11]:
print(Fe3GeTe2.to_magnopy(comments=False))

GROGU INFORMATION
Hamiltonian convention
Double counting      true
Normalized spins     true
Intra-atomic factor  +1
Exchange factor      +0.5
Cell (Ang)
3.791001511088653242e+00 0.000000000000000000e+00 0.000000000000000000e+00
-1.895500755544326621e+00 3.283103614407953064e+00 0.000000000000000000e+00
0.000000000000000000e+00 0.000000000000000000e+00 2.057000819825037041e+01
Magnetic sites
Number of sites 2
Name x (Ang) y (Ang) z (Ang) s sx sy sz
3Fe(l:2) 1.981762848288204e-06 -9.134332829322269e-08 11.653315176909826 2.011394466844794 6.447254726229378e-05 -0.0005169351977350883 2.011394399384453
4Fe(l:2) 1.990383996147767e-06 -1.089291427685217e-07 8.916695475519404 2.0114468870381006 -6.171573052964394e-05 0.000512053681777345 2.0114468209146024
Intra-atomic anisotropy tensor (meV)
--------------------------------------------------------------------------------
3Fe(l:2)
Matrix
    0.19805059848871515 -2.5314299080536617e-05 0.00039210954835139367
    -2.5314299080536617e-05 1.0811

More information can be found in the Tutorials.